In [3]:
# Install albumentations
%pip install albumentations

   ---------------------------------------- 2.0/2.0 MB 6.5 MB/s eta 0:00:00
   ---------------------------------------- 15.8/15.8 MB 5.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.17
    Uninstalling pydantic-1.10.17:
      Successfully uninstalled pydantic-1.10.17
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi-utils 0.2.1 requires pydantic<2.0,>=1.0, but you have pydantic 2.10.5 which is incompatible.
mlserver 1.5.0 requires pydantic<2.0.0, but you have pydantic 2.10.5 which is incompatible.
mlserver-mlflow 1.5.0 requires pydantic<2.0.0, but you have pydantic 2.10.5 which is incompatible.
sqlmodel 0.0.8 requires pydantic<2.0.0,>=1.8.2, but you have pydantic 2.10.5 which is incompatible.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.4 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.
zenml 0.58.2 requires pydantic<1.11,>=1.9.0, but you have pydantic 2.10.5 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice]

In [5]:
%pip install --upgrade numpy

  Using cached numpy-2.2.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 12.9/12.9 MB 6.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.14.1 requires numpy<2, but you have numpy 2.2.1 which is incompatible.
mlserver 1.5.0 requires pydantic<2.0.0, but you have pydantic 2.10.5 which is incompatible.
mlserver-mlflow 1.5.0 requires pydantic<2.0.0, but you have pydantic 2.10.5 which is incompatible.
pyarrow 15.0.2 requires numpy<2,>=1.16.6, but you have numpy 2.2.1 which is incompatible.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.2.1 which is incompatible.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 2.2.1 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.
zenml 0.58.2 requires pydantic<1.11,>=1.9.0, but you have pydantic 2.10.5 which is incompatible.

[notice] A new release of pip is available:

In [8]:
%pip uninstall numpy scipy pydantic

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50
import numpy as np

class CenterNetDental(nn.Module):
    def __init__(self, num_classes=1, output_size=(128, 128)):
        super(CenterNetDental, self).__init__()
        
        # ResNet50 backbone with pretrained weights
        backbone = resnet50(pretrained=True)
        self.backbone = nn.Sequential(*list(backbone.children())[:-2])
        
        # Upsampling layers
        self.deconv_layers = nn.Sequential(
            nn.ConvTranspose2d(2048, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        
        # Head layers for different tasks
        self.heatmap_head = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, num_classes, kernel_size=1)
        )
        
        self.wh_head = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 2, kernel_size=1)
        )
        
        self.offset_head = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 2, kernel_size=1)
        )
        
        self.output_size = output_size

    def forward(self, x):
        # Backbone feature extraction
        features = self.backbone(x)
        
        # Upsampling
        features = self.deconv_layers(features)
        
        # Get predictions
        heatmaps = self.heatmap_head(features)
        wh = self.wh_head(features)
        offset = self.offset_head(features)
        
        # Apply sigmoid to heatmaps
        heatmaps = torch.sigmoid(heatmaps)
        
        return heatmaps, wh, offset

def gaussian2D(shape, sigma=1):
    m, n = [(ss - 1.) / 2. for ss in shape]
    y, x = np.ogrid[-m:m+1, -n:n+1]
    
    h = np.exp(-(x * x + y * y) / (2 * sigma * sigma))
    h[h < np.finfo(h.dtype).eps * h.max()] = 0
    return h

def draw_gaussian(heatmap, center, radius, k=1):
    diameter = 2 * radius + 1
    gaussian = gaussian2D((diameter, diameter), sigma=diameter / 6)
    
    x, y = int(center[0]), int(center[1])
    
    height, width = heatmap.shape[0:2]
    
    left, right = min(x, radius), min(width - x, radius + 1)
    top, bottom = min(y, radius), min(height - y, radius + 1)
    
    masked_heatmap = heatmap[y - top:y + bottom, x - left:x + right]
    masked_gaussian = gaussian[radius - top:radius + bottom, radius - left:radius + right]
    
    if min(masked_gaussian.shape) > 0 and min(masked_heatmap.shape) > 0:
        np.maximum(masked_heatmap, masked_gaussian * k, out=masked_heatmap)
    
    return heatmap

class DentalDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, annotations, output_size=(128, 128), transform=None):
        self.image_paths = image_paths
        self.annotations = annotations
        self.output_size = output_size
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load image
        img = load_image(self.image_paths[idx])  # Implement load_image based on your needs
        
        if self.transform:
            img = self.transform(img)
        
        # Get annotations for this image
        boxes = self.annotations[idx]
        
        # Create target maps
        heatmap = np.zeros((1, self.output_size[0], self.output_size[1]))
        wh = np.zeros((2, self.output_size[0], self.output_size[1]))
        offset = np.zeros((2, self.output_size[0], self.output_size[1]))
        reg_mask = np.zeros((self.output_size[0], self.output_size[1]))
        
        # Process each box
        for box in boxes:
            x1, y1, x2, y2 = box
            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2
            w = x2 - x1
            h = y2 - y1
            
            # Scale to output size
            cx = cx * self.output_size[1] / img.shape[1]
            cy = cy * self.output_size[0] / img.shape[0]
            w = w * self.output_size[1] / img.shape[1]
            h = h * self.output_size[0] / img.shape[0]
            
            # Draw gaussian
            draw_gaussian(heatmap[0], (cx, cy), radius=3)
            
            # Add width/height
            wh[0, int(cy), int(cx)] = w
            wh[1, int(cy), int(cx)] = h
            
            # Add offset
            offset[0, int(cy), int(cx)] = cx - int(cx)
            offset[1, int(cy), int(cx)] = cy - int(cy)
            
            # Add reg mask
            reg_mask[int(cy), int(cx)] = 1
            
        return {
            'image': torch.FloatTensor(img),
            'heatmap': torch.FloatTensor(heatmap),
            'wh': torch.FloatTensor(wh),
            'offset': torch.FloatTensor(offset),
            'reg_mask': torch.FloatTensor(reg_mask)
        }

def focal_loss(pred, gt):
    pos_inds = gt.eq(1).float()
    neg_inds = gt.lt(1).float()
    
    neg_weights = torch.pow(1 - gt, 4)
    
    loss = 0
    
    pos_loss = torch.log(pred) * torch.pow(1 - pred, 2) * pos_inds
    neg_loss = torch.log(1 - pred) * torch.pow(pred, 2) * neg_weights * neg_inds
    
    num_pos = pos_inds.float().sum()
    pos_loss = pos_loss.sum()
    neg_loss = neg_loss.sum()
    
    if num_pos == 0:
        loss = loss - neg_loss
    else:
        loss = loss - (pos_loss + neg_loss) / num_pos
        
    return loss

def train_model(model, train_loader, optimizer, epoch):
    model.train()
    
    for batch_idx, batch in enumerate(train_loader):
        images = batch['image']
        heatmap_target = batch['heatmap']
        wh_target = batch['wh']
        offset_target = batch['offset']
        reg_mask = batch['reg_mask']
        
        optimizer.zero_grad()
        
        heatmap_pred, wh_pred, offset_pred = model(images)
        
        # Calculate losses
        heatmap_loss = focal_loss(heatmap_pred, heatmap_target)
        wh_loss = F.l1_loss(wh_pred * reg_mask.unsqueeze(1), 
                           wh_target * reg_mask.unsqueeze(1),
                           reduction='sum') / (reg_mask.sum() + 1e-4)
        offset_loss = F.l1_loss(offset_pred * reg_mask.unsqueeze(1),
                               offset_target * reg_mask.unsqueeze(1),
                               reduction='sum') / (reg_mask.sum() + 1e-4)
        
        loss = heatmap_loss + wh_loss + offset_loss
        
        loss.backward()
        optimizer.step()

In [4]:
import torch
import numpy as np
import cv2
from torchvision import transforms
import albumentations as A
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt

class DentalAugmentation:
    def __init__(self, p=0.5):
        self.transform = A.Compose([
            # Intensity adjustments to simulate different exposure levels
            A.RandomBrightnessContrast(
                brightness_limit=0.2,
                contrast_limit=0.2,
                p=p
            ),
            # Simulate different X-ray angles
            A.ShiftScaleRotate(
                shift_limit=0.1,
                scale_limit=0.1,
                rotate_limit=15,
                border_mode=cv2.BORDER_CONSTANT,
                p=p
            ),
            # Add noise to simulate different X-ray qualities
            A.GaussNoise(
                var_limit=(10.0, 50.0),
                p=p
            ),
            # Adjust gamma to simulate different tissue densities
            A.RandomGamma(
                gamma_limit=(80, 120),
                p=p
            ),
            # Elastic deformation to simulate tissue variation
            A.ElasticTransform(
                alpha=1,
                sigma=50,
                alpha_affine=50,
                border_mode=cv2.BORDER_CONSTANT,
                p=p
            )
        ], bbox_params=A.BboxParams(
            format='pascal_voc',
            label_fields=['labels']
        ))
    
    def __call__(self, image, boxes=None):
        if boxes is None:
            transformed = self.transform(image=image)
            return transformed['image']
        
        # Prepare boxes for augmentation
        labels = ['tooth'] * len(boxes)
        transformed = self.transform(
            image=image,
            bboxes=boxes,
            labels=labels
        )
        
        return transformed['image'], transformed['bboxes']

def post_process_detections(heatmap, wh, offset, confidence_threshold=0.3, nms_threshold=0.5):
    """
    Process CenterNet outputs to get final bounding box predictions
    """
    batch_size = heatmap.shape[0]
    height = heatmap.shape[2]
    width = heatmap.shape[3]
    num_classes = heatmap.shape[1]
    
    # Get peaks in the heatmap
    heatmap = torch.nn.functional.max_pool2d(
        heatmap,
        kernel_size=3,
        padding=1,
        stride=1
    )
    
    scores, indices = torch.max(heatmap, dim=1)
    scores = scores.view(batch_size, -1)
    topk_scores, topk_inds = torch.topk(scores, k=100)
    
    topk_ys = (topk_inds / width).int().float()
    topk_xs = (topk_inds % width).int().float()
    
    # Get width and height predictions
    wh = wh.view(batch_size, 2, -1)
    offset = offset.view(batch_size, 2, -1)
    
    topk_wh = torch.gather(wh, 2, 
        topk_inds.unsqueeze(1).expand(batch_size, 2, 100))
    topk_offset = torch.gather(offset, 2,
        topk_inds.unsqueeze(1).expand(batch_size, 2, 100))
    
    # Calculate bounding box coordinates
    boxes = torch.zeros_like(topk_wh)
    boxes[:, 0] = topk_xs + topk_offset[:, 0]  # x1
    boxes[:, 1] = topk_ys + topk_offset[:, 1]  # y1
    
    # Convert center to corners
    boxes[:, 0] -= topk_wh[:, 0] / 2  # x1
    boxes[:, 1] -= topk_wh[:, 1] / 2  # y1
    boxes[:, 0] += topk_wh[:, 0]      # x2
    boxes[:, 1] += topk_wh[:, 1]      # y2
    
    # Filter by confidence
    mask = topk_scores > confidence_threshold
    boxes = boxes[mask]
    scores = topk_scores[mask]
    
    # Apply NMS
    keep = nms(boxes, scores, nms_threshold)
    boxes = boxes[keep]
    scores = scores[keep]
    
    return boxes, scores

def evaluate_model(model, val_loader, device):
    """
    Evaluate model performance using mean Average Precision (mAP)
    """
    model.eval()
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(device)
            target_heatmaps = batch['heatmap']
            
            # Get predictions
            heatmaps, wh, offset = model(images)
            boxes, scores = post_process_detections(
                heatmaps, wh, offset
            )
            
            # Convert predictions and targets to common format
            all_predictions.extend(scores.cpu().numpy())
            all_targets.extend(target_heatmaps.view(-1).cpu().numpy())
    
    # Calculate mAP
    mAP = average_precision_score(all_targets, all_predictions)
    return mAP

def visualize_predictions(image, boxes, scores, threshold=0.5):
    """
    Visualize detection results on the image
    """
    image = image.copy()
    
    # Filter predictions by confidence
    mask = scores > threshold
    boxes = boxes[mask]
    scores = scores[mask]
    
    # Draw boxes
    for box, score in zip(boxes, scores):
        x1, y1, x2, y2 = box.int()
        cv2.rectangle(
            image,
            (x1, y1), (x2, y2),
            color=(0, 255, 0),
            thickness=2
        )
        cv2.putText(
            image,
            f'{score:.2f}',
            (x1, y1 - 5),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 255, 0),
            2
        )
    
    return image

def train_with_validation(model, train_loader, val_loader, 
                         optimizer, num_epochs, device):
    """
    Training loop with validation
    """
    best_map = 0
    train_losses = []
    val_maps = []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        epoch_losses = []
        
        for batch in train_loader:
            images = batch['image'].to(device)
            heatmap_target = batch['heatmap'].to(device)
            wh_target = batch['wh'].to(device)
            offset_target = batch['offset'].to(device)
            reg_mask = batch['reg_mask'].to(device)
            
            optimizer.zero_grad()
            
            heatmap_pred, wh_pred, offset_pred = model(images)
            
            # Calculate losses
            loss = calculate_total_loss(
                heatmap_pred, heatmap_target,
                wh_pred, wh_target,
                offset_pred, offset_target,
                reg_mask
            )
            
            loss.backward()
            optimizer.step()
            
            epoch_losses.append(loss.item())
        
        # Validation
        mAP = evaluate_model(model, val_loader, device)
        
        # Save best model
        if mAP > best_map:
            best_map = mAP
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'mAP': mAP,
            }, 'best_model.pth')
        
        # Record metrics
        train_losses.append(np.mean(epoch_losses))
        val_maps.append(mAP)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Training Loss: {train_losses[-1]:.4f}')
        print(f'Validation mAP: {mAP:.4f}')
        
        # Plot metrics
        plot_metrics(train_losses, val_maps)

def plot_metrics(train_losses, val_maps):
    """
    Plot training metrics
    """
    plt.figure(figsize=(12, 4))
    
    # Plot training loss
    plt.subplot(1, 2, 1)
    plt.plot(train_losses)
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
    # Plot validation mAP
    plt.subplot(1, 2, 2)
    plt.plot(val_maps)
    plt.title('Validation mAP')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    
    plt.tight_layout()
    plt.savefig('training_metrics.png')
    plt.close()

def calculate_total_loss(heatmap_pred, heatmap_target,
                        wh_pred, wh_target,
                        offset_pred, offset_target,
                        reg_mask):
    """
    Calculate combined loss for all predictions
    """
    heatmap_loss = focal_loss(heatmap_pred, heatmap_target)
    wh_loss = F.l1_loss(
        wh_pred * reg_mask.unsqueeze(1),
        wh_target * reg_mask.unsqueeze(1),
        reduction='sum'
    ) / (reg_mask.sum() + 1e-4)
    offset_loss = F.l1_loss(
        offset_pred * reg_mask.unsqueeze(1),
        offset_target * reg_mask.unsqueeze(1),
        reduction='sum'
    ) / (reg_mask.sum() + 1e-4)
    
    return heatmap_loss + wh_loss + offset_loss

In [4]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
import cv2
import os
from typing import Dict, List, Tuple
from torchvision import transforms

class DentalXrayDataset(Dataset):
    def __init__(self, 
                 csv_file: str,
                 img_dir: str,
                 output_size: Tuple[int, int] = (128, 128),
                 transform=None):
        """
        Args:
            csv_file (str): Path to CSV file with annotations
            img_dir (str): Directory with images
            output_size (tuple): Size of output heatmap (height, width)
            transform: Optional transform to be applied on images
        """
        # Read CSV and process data
        self.df = pd.read_csv(csv_file, header=None, 
                             names=['filename', 'width', 'height', 'class', 
                                   'xmin', 'ymin', 'xmax', 'ymax'])
        self.img_dir = img_dir
        self.transform = transform
        self.output_size = output_size
        
        # Create class mapping
        self.class_map = {
            'Class 0- No endodontic treatment': 0,
            'Class 1- complete endodontic treatment': 1,
            'Class 2- incomplete endodontic treatment': 2,
            'cavity': 3
        }
        
        # Group annotations by filename
        self.image_files = self.df['filename'].unique()
        self.grouped_annotations = self.df.groupby('filename')
    
    def __len__(self):
        return len(self.image_files)
    
    def _get_boxes_and_classes(self, filename: str) -> Tuple[np.ndarray, List[int]]:
        """Get all bounding boxes and classes for an image"""
        annotations = self.grouped_annotations.get_group(filename)
        boxes = annotations[['xmin', 'ymin', 'xmax', 'ymax']].values
        classes = [self.class_map[cls] for cls in annotations['class']]
        return boxes, classes
    
    def _create_heatmap(self, 
                       boxes: np.ndarray, 
                       classes: List[int], 
                       original_size: Tuple[int, int]) -> np.ndarray:
        """Create heatmap for CenterNet"""
        num_classes = len(self.class_map)
        heatmap = np.zeros((num_classes, *self.output_size))
        
        for box, cls_id in zip(boxes, classes):
            # Scale box to output size
            x1, y1, x2, y2 = box
            x1 = x1 * self.output_size[1] / original_size[1]
            y1 = y1 * self.output_size[0] / original_size[0]
            x2 = x2 * self.output_size[1] / original_size[1]
            y2 = y2 * self.output_size[0] / original_size[0]
            
            # Calculate center point
            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2
            
            # Create gaussian peak
            draw_gaussian(heatmap[cls_id], (int(cx), int(cy)), radius=3)
            
        return heatmap
    
    def _create_wh_maps(self, 
                       boxes: np.ndarray, 
                       original_size: Tuple[int, int]) -> np.ndarray:
        """Create width-height maps"""
        wh = np.zeros((2, *self.output_size))
        reg_mask = np.zeros(self.output_size)
        
        for box in boxes:
            x1, y1, x2, y2 = box
            # Scale to output size
            x1 = x1 * self.output_size[1] / original_size[1]
            y1 = y1 * self.output_size[0] / original_size[0]
            x2 = x2 * self.output_size[1] / original_size[1]
            y2 = y2 * self.output_size[0] / original_size[0]
            
            # Calculate center point
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            
            # Calculate width and height
            w = x2 - x1
            h = y2 - y1
            
            # Add to maps
            wh[0, cy, cx] = w
            wh[1, cy, cx] = h
            reg_mask[cy, cx] = 1
            
        return wh, reg_mask
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        # Get image filename
        img_file = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_file)
        
        # Read image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        original_size = (image.shape[0], image.shape[1])
        
        # Get boxes and classes
        boxes, classes = self._get_boxes_and_classes(img_file)
        
        # Create target maps
        heatmap = self._create_heatmap(boxes, classes, original_size)
        wh, reg_mask = self._create_wh_maps(boxes, original_size)
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        # Convert to tensors
        return {
            'image': torch.FloatTensor(image),
            'heatmap': torch.FloatTensor(heatmap),
            'wh': torch.FloatTensor(wh),
            'reg_mask': torch.FloatTensor(reg_mask),
            'boxes': torch.FloatTensor(boxes),
            'classes': torch.LongTensor(classes)
        }

def create_data_loaders(csv_file: str,
                       img_dir: str,
                       batch_size: int = 4,
                       train_split: float = 0.8):
    """Create train and validation data loaders"""
    # Define transforms
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    # Create dataset
    dataset = DentalXrayDataset(
        csv_file=csv_file,
        img_dir=img_dir,
        transform=transform
    )
    
    # Split dataset
    train_size = int(train_split * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size]
    )
    
    # Create data loaders
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4
    )
    
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4
    )
    
    return train_loader, val_loader

In [10]:
# 1. Set up paths
csv_file = "train\_annotations.csv"
img_dir = "train"

# 2. Create data loaders
train_loader, val_loader = create_data_loaders(
    csv_file=csv_file,
    img_dir=img_dir,
    batch_size=4
)

# 3. Create and train model
model = CenterNetDental(
    num_classes=4,  # 3 treatment classes + cavity
    output_size=(128, 128)
)

# 4. Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Train with validation
train_with_validation(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    num_epochs=50,
    device=device
)

d:\PE19\FYP\CenterNet Codes\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\PE19\FYP\CenterNet Codes\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\ajani/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


NameError: name 'train_with_validation' is not defined